In [1]:
from qiskit.transpiler.passes.routing.sabre_swap_v020  import SabreSwap as SabreSwap_v020
from qiskit.transpiler.passes.layout.sabre_layout_v020 import SabreLayout as SabreLayout_v020
from qiskit.transpiler.passes.routing.sabre_swap_v020_dive import SabreSwap as SabreSwap_test


from ulti.circuits import get_circuit_list, sort_circuits_by_depth
from ulti.coupling_maps import file_to_coupling_map
from ulti.pass_managers import build_pm
import time

In [2]:
qc_name = "size5_depth_1-10"
cp_name = "hex_7"
cm = file_to_coupling_map(f'coupling_maps/{cp_name}.txt')
seed = 42

# variables to change
lookahead = 5
fast_layout = True

# pass manager for Sabre v0.20
rp_v020 = SabreSwap_v020
lp_v020 = SabreLayout_v020
pm_v020 = build_pm(rp_v020, lp_v020, cm, seed=seed, fast_layout=fast_layout)

# pass manager for Sabre test
rp_test = SabreSwap_test
pm_test = build_pm(rp_test, lp_v020, cm, seed=seed, fast_layout=fast_layout)

qc_list = get_circuit_list([f'circuits/{qc_name}/'])
qc_list = sort_circuits_by_depth(qc_list)

# test the 7th and 8th circuits
#qc_list = qc_list[6:8]

In [3]:
print(qc_list)

[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1683462d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16831f910>]


In [4]:
# draw each of the circuits
for i, qc in enumerate(qc_list):
    print(f"Circuit {i+1}:\n")
    print(qc.decompose().draw(output='text'))
    print("\n")
    print("-"*100)

Circuit 1:

                                 ┌────────────┐              ┌────────────┐»
q_0: ────────────────────────────┤1           ├──────────────┤0           ├»
                   ┌────────────┐│            │              │            │»
q_1: ──────────────┤0           ├┤            ├──────────────┤            ├»
     ┌────────────┐│            ││  Unitary_2 │┌────────────┐│  Unitary_4 │»
q_2: ┤0           ├┤            ├┤            ├┤0           ├┤            ├»
     │  Unitary_1 ││  Unitary_0 ││            ││            ││            │»
q_3: ┤1           ├┤            ├┤0           ├┤  Unitary_3 ├┤1           ├»
     └────────────┘│            │└────────────┘│            │└────────────┘»
q_4: ──────────────┤1           ├──────────────┤1           ├──────────────»
                   └────────────┘              └────────────┘              »
«     ┌────────────┐┌────────────┐┌─────────────┐               ┌─────────────┐»
«q_0: ┤1           ├┤1           ├┤1            ├───────────

In [5]:
depths_20 = []
depths_test = []

# Loop through the circuits and run them through the pass managers
for i, qc in enumerate(qc_list):
    qc = qc.decompose()


    # Transpile using v0.20
    start_20 = time.time()
    qc_tr_20 = pm_v020.run(qc)
    end_20 = time.time()
    duration_20 = end_20 - start_20
    depth_20 = qc_tr_20.decompose(["swap"]).depth()
    depths_20.append(depth_20)

    # Transpile using v0.20 with lookahead
    start_test = time.time()
    qc_tr_test = pm_test.run(qc)
    end_test = time.time()
    duration_test = end_test - start_test
    depth_test = qc_tr_test.decompose(["swap"]).depth()
    depths_test.append(depth_test)
    # Print the depth and duration for each transpilation
    print(f"{'Depth after transpiled v0.20:':<60}{depth_20:<10}{'Time taken:':<20}{duration_20:.5f} seconds")
    print(f"{'Depth after transpiled v0.20 with lookahead:':<60}{depth_test:<10}{'Time taken:':<20}{duration_test:.5f} seconds")

    print("-"*100)

# Print the average depth and duration for each transpilation
print(f"{'Average depth after transpiled v0.20:':<60}{sum(depths_20)/len(depths_20):<10.5f}")
print(f"{'Average depth after transpiled v0.20 with lookahead:':<60}{sum(depths_test)/len(depths_test):<10.5f}")


Depth after transpiled v0.20:                               19        Time taken:         0.01989 seconds
Depth after transpiled v0.20 with lookahead:                19        Time taken:         0.01696 seconds
----------------------------------------------------------------------------------------------------
Depth after transpiled v0.20:                               43        Time taken:         0.03750 seconds
Depth after transpiled v0.20 with lookahead:                43        Time taken:         0.01900 seconds
----------------------------------------------------------------------------------------------------
Average depth after transpiled v0.20:                       31.00000  
Average depth after transpiled v0.20 with lookahead:        31.00000  
